# Notebook avec prompts éditoriaux avec l'API Gemini de Google


### ÉTAPE 1: Installation des bibliothèques nécessaires
Installer le SDK Python de Google AI avec pip

In [ ]:
!pip install -q google-generativeai

### ÉTAPE 2: Importation des modules nécessaires


In [ ]:
import os
import json
import io
import google.generativeai as genai
from IPython.display import display, Markdown
from google.colab import files, userdata


### ÉTAPE 3: Configuration de l'authentification API

Récupérer la clé API de manière sécurisée depuis les secrets de Colab
Vous devez d'abord enregistrer votre clé dans l'outil de secrets de Colab

In [ ]:
api_key = userdata.get('GEMINI_API_KEY')

print('API KEY is OK')

# Alternative moins sécurisée : définir directement la clé API (déconseillé)
# api_key = "VOTRE_CLÉ_API"  # Remplacez par votre clé API réelle

API KEY is OK


### ÉTAPE 4: Configuration de la bibliothèque d'IA générative

Initialiser la bibliothèque avec votre clé API

In [ ]:
genai.configure(api_key=api_key)


def get_gemini_response(prompt_text, model="gemini-1.5-pro"):
    """
    Obtenir une réponse du modèle Gemini.

    Args:
        prompt_text (str): Le texte de la requête à envoyer
        model (str): Le modèle Gemini à utiliser

    Returns:
        La réponse générée par l'API
    """
    # Initialiser le modèle
    model = genai.GenerativeModel(model)

    # Générer le contenu
    response = model.generate_content(prompt_text)

    return response

### Étape 4: Création du prompt
Créer le prompt avec des variables

In [ ]:
# Définition des variables directement dans le code principal
language = "french"

# CONTENU ARTICLE V1

# content = """
# La cinquantaine de pays qui viennent en aide à l'Ukraine en guerre ont promis vendredi de renforcer leur aide militaire,
# au moment où les États-Unis ont réduit leur soutien. "Aujourd'hui, j'annonce une enveloppe de 350
# millions de livres (407 millions d'euros)", a déclaré à Bruxelles le ministre britannique de la Défense John Healey.
# Récit d'Eliott Samuel et David Gilberg.
# """

# CONTENU ARTICLE V2
content = """
Un butin et une contre-enquête. C'est ce que le musée du Quai Branly offre à ses visiteurs à partir de mercredi 15 avril,
avec une exposition consacrée à la "mission ethnographique et linguistique Dakar-Djibouti".
Une expédition qui, dans les années 1930, a servi à s'approprier des milliers d'objets du patrimoine culturel africain.
Le grand musée parisien des "arts premiers" s'est ainsi penché sur les méthodes de travail et d'acquisition
de cette mission pluridisciplinaire, ouvrant ses fonds et ses archives à des chercheurs – notamment africains – appelés à s'interroger
sur les procédés de l'équipe expéditionnaire.
Entre 1931 et 1933, à travers 14 pays du continent africain – tous sous domination coloniale,
à l’exception de l’Éthiopie –, tous les moyens sont bons pour prendre possession des quelque 3 000
objets expédiés ensuite au Musée d’ethnographie du Trocadéro, à Paris, l'ancêtre du Musée de l’Homme.
L’objectif annoncé de cette "mission ethnographique" était de documenter des cultures africaines
supposées en voie de disparition sous l’effet de la colonisation.
Mais sous couvert de recherche scientifique, le groupe chapeauté par l’ethnologue Marcel Griaule et
l'écrivain Michel Leiris se livre à un pillage en bonne et due forme. Les méthodes de l'expédition soulèvent des questions éthiques,
notamment sur les relations de pouvoir entre colonisateurs et colonisés.
"""

# PROMPT_1 : Depuis un {content} en {language} , générer 3 titres et 5 mots-clés en {language}
# prompt_example = f"""
#     You are a smart and intelligent journalist. Craft three compelling and unique
#     titles for an online post about the topic given in the content: '{content}' in '{language}'.
#     Ensure to incorporate SEO best practices by including the most common and relevant keywords from
#     the content in each title. For each proposal, print only the result in a Python dictionary object
#     with 'title' as a string and 'keywords' as a list. Do not print anything else. Include all three results into a Python list object
#     like defined below. Output Format:[{{"title": "The value of the title", "keywords": ["keyword1", "keyword2", "keyword3"]}},
#     {{"title": "The value of the title", "keywords": ["keyword1", "keyword2", "keyword3"]}},
#     {{"title": "The value of the title", "keywords":["keyword1", "keyword2", "keyword3"]}}]
#     """

# PROMPT_2 : Générer le résumé dans une {language}, sans longueur spécifique
# prompt_example = f"""
# Given the input text in '{language}': Input: '{content}' Produce a summary of the text in '{language}'.
# The summary must be in the same language from the original text in '{language}'.
# Captures the main ideas and key points of the text.
# Summary Does not include verbatim sentences from the original text. For the proposal,
# print only the result in a Python dictionary object with the summary as a string.
# Include the all result into a Python list object like define below. Output Format: [ {{"summary": "The summary of the content"}}]
# """

# PROMPT_3 : Générer le résumé dans une {language} avec une longueur spécifique {summary_length}
# summary_length = 5
# prompt_example = f"""
# Given the input text in '{language}': Input: '{content}' Produce a '{summary_length}' sentences length summary of the text in '{language}'.
# The summary must be in the same language from the original text in '{language}'.
# Captures the main ideas and key points of the text. Summary Does not include verbatim sentences from the original text.
# For the proposal, print only the result in a Python dictionary object with the summary as a string and the summary_length keywords as a integer.
# Include the all result into a Python list object like define below.
# Output Format: [ {{"summary": "The summary of the content", "summary_length": summary_length_number }}]
# """

# PROMPT_4 : Depuis un contenu en  {language} , générer 3 tweets de 280 caractéres et 3 hashtags en  {language}
prompt_example = f"""
You are a smart and intelligent community manager.
Craft three compelling messages of 280 characters each for an online post in {language} about the topic given in the content in {language}.
Ensure to incorporate Social Media Optimization (SMO) best practices by including the most common keywords from the content.
For each proposal in {language}, print only the result in a Python dictionary object with 'message' as a string and 'hashtags' as a list of hashtags.
In the list of hashtags, for each hashtag, do not forget to add the sign "#" in front of it e.g. "hashtags": ["#hashtag1", "#hashtag2", "#hashtag3"].
Include all results into a Python list object as defined below.
Output Format:[{{"message": "The value of the message", "hashtags": ["#hashtag1", "#hashtag2", "#hashtag3"]}}, {{"message": "The value of the message", "hashtags": ["#hashtag1", "#hashtag2", "#hashtag3"]}}, {{"message": "The value of the message", "hashtags": ["#hashtag1", "#hashtag2", "#hashtag3"]}}] Content: '{content}'
"""


print(f"Requête chargée: {prompt_example}\n")


Requête chargée: 
You are a smart and intelligent community manager.
Craft three compelling messages of 280 characters each for an online post in french about the topic given in the content in french.
Ensure to incorporate Social Media Optimization (SMO) best practices by including the most common keywords from the content.
For each proposal in french, print only the result in a Python dictionary object with 'message' as a string and 'hashtags' as a list of hashtags.
In the list of hashtags, for each hashtag, do not forget to add the sign "#" in front of it e.g. "hashtags": ["#hashtag1", "#hashtag2", "#hashtag3"].
Include all results into a Python list object as defined below.
Output Format:[{"message": "The value of the message", "hashtags": ["#hashtag1", "#hashtag2", "#hashtag3"]}, {"message": "The value of the message", "hashtags": ["#hashtag1", "#hashtag2", "#hashtag3"]}, {"message": "The value of the message", "hashtags": ["#hashtag1", "#hashtag2", "#hashtag3"]}] Content: '
Un but

### Étape 5: Réponse de Gemini
La réponse est au format défini dans le prompt


In [ ]:
print("Envoi à l'API Gemini...\n")

# Obtenir la réponse de Gemini
try:
    # Appel à l'API Gemini
    response = get_gemini_response(prompt_example)

    # Afficher la réponse
    print("Réponse de Gemini:")
    print("-" * 50)
    display(Markdown(response.text))
    print("-" * 50)
except Exception as e:
    # Gestion des erreurs
    print(f"Erreur: {e}")
    print("Si vous voyez une erreur de clé API, assurez-vous de configurer correctement votre clé API.")
    print("Instructions: Allez sur Google AI Studio pour obtenir une clé API")
    print("Ensuite dans Colab, cliquez sur l'icône 🔑 dans la barre latérale gauche pour l'enregistrer comme GEMINI_API_KEY")


Envoi à l'API Gemini...

